In [213]:
import math 
import json 

import geopandas as gpd
import h3
import pandas as pd
from shapely.geometry.polygon import Polygon

Загрузим данные.

In [214]:
gdf = gpd.read_file('data/buildings.gpkg')

In [215]:
gdf.columns

Index(['cad_num', 'cost_value', 'year_built', 'floors', 'materials', 'purpose',
       'area', 'rayon', 'geometry'],
      dtype='object')

Определим гексы, к которым относятся здания, опираясь на центроиду. 

In [216]:
def define_hex(x):
    center = x.centroid
    return h3.geo_to_h3(center.x, center.y, 7)

In [217]:
gdf['h'] = gdf.geometry.apply(define_hex)

In [218]:
len(gdf['h'].unique())

31

При разрешении в 7 получается 31 гексагон. 

In [219]:
gdf['h'].value_counts(normalize=True)

h
872cce8c9ffffff    0.098760
872cce164ffffff    0.095778
872cce8cbffffff    0.088263
872cce8d9ffffff    0.086772
872cce166ffffff    0.066734
872cce8caffffff    0.063633
872cceb94ffffff    0.060472
872cce8c8ffffff    0.056119
872cceb96ffffff    0.049380
872ccebb2ffffff    0.043714
872ccebb3ffffff    0.042283
872cce160ffffff    0.033993
872cce165ffffff    0.031966
872cce8dbffffff    0.028865
872cce8cdffffff    0.026121
872cce175ffffff    0.025763
872cce8d8ffffff    0.018667
872cce8daffffff    0.014850
872cce174ffffff    0.014074
872ccebb6ffffff    0.011033
872cce176ffffff    0.010496
872cce8ddffffff    0.009721
872ccebb0ffffff    0.006620
872cce170ffffff    0.006143
872ccebb1ffffff    0.005069
872cce8ceffffff    0.003698
872cceb86ffffff    0.000417
872cceb92ffffff    0.000239
872cce129ffffff    0.000179
872cceb90ffffff    0.000119
872cceb95ffffff    0.000060
Name: proportion, dtype: float64

Создадим новый геодатафрейм - с данными по каждому гексу. И первым делом отбросим те гексы, где менее 100 зданий - слишком уж нерепрезентативно по такому маленькому количеству данных делать статистические выводы. 

Посчитаем количество домов, которые входят в гекс, общую кадастровую стоимость недвижимости, средний и максимальный год постройки здания, среднее и максимальное количество этажей от максимального количества этажей (см. подготоку данных) в зданиях. 

In [220]:
hexes = pd.pivot_table(data=gdf, index='h', aggfunc={
    'cad_num': 'count',
    'cost_value': 'sum', 
    'year_built': ['mean', 'max'],
    'floors': ['mean', 'max'], 
    
})

hexes = hexes[hexes[('cad_num', 'count')] >= 100]
hexes[('floors', 'max')] = hexes[('floors', 'max')].astype(int)
hexes[('floors', 'mean')] = hexes[('floors', 'mean')].apply(lambda x: int(math.ceil(x)))
hexes[('year_built', 'max')] = hexes[('year_built', 'max')].astype(int)
hexes[('year_built', 'mean')] = hexes[('year_built', 'mean')].apply(lambda x: int(math.floor(x)))

В итоге получается вот такой набор цифр: 

In [221]:
hexes

cad_num    cost_value floors      year_built      
                  count           sum    max mean        max  mean
h                                                                 
872cce160ffffff     570  3.864451e+11     18    5       2018  1946
872cce164ffffff    1606  1.480858e+12     19    4       2021  1923
872cce165ffffff     536  4.408207e+11     39    5       2020  1949
872cce166ffffff    1119  1.067231e+12     31    5       2021  1940
872cce170ffffff     103  3.048329e+10     14    3       2014  1968
872cce174ffffff     236  5.049558e+11     80    7       2022  1964
872cce175ffffff     432  3.011344e+11     22    5       2021  1954
872cce176ffffff     176  1.529099e+11     88    6       2020  1974
872cce8c8ffffff     941  7.062743e+11     35    5       2021  1931
872cce8c9ffffff    1656  8.789334e+11     33    4       2023  1921
872cce8caffffff    1067  9.410942e+11     22    4       2022  1934
872cce8cbffffff    1480  1.179341e+12     15    4       2016  1911
872cce8cdffffff     438  3.148997e+11     24    5       2016  1936
872cce8d8ffffff     313  6.374254e+11     20    5       2021  1956
872cce8d9ffffff    1455  1.580587e+12     26    4       2021  1928
872cce8daffffff     249  1.872230e+11     14    4       2021  1940
872cce8dbffffff     484  6.020069e+11     34    5       2023  1941
872cce8ddffffff     163  1.289526e+11     23    4       2021  1933
872cceb94ffffff    1014  4.504654e+11     22    4       2016  1940
872cceb96ffffff     828  6.377086e+11     29    5       2022  1945
872ccebb0ffffff     111  7.880786e+10     25    6       2020  1974
872ccebb2ffffff     733  4.544069e+11     23    4       2022  1938
872ccebb3ffffff     709  2.818650e+11     22    4       2021  1945
872ccebb6ffffff     185  1.077797e+11     19    5       2015  1956

Теперь переведём эти данные в геоформат, чтобы была возможность оперировать геометрией гексов. 

In [222]:
hexes_geo = pd.DataFrame()
hexes['id'] = hexes.index
hexes_geo['id'] = hexes[('id', '')]
hexes_geo['buildings_count'] = hexes[('cad_num', 'count')]
hexes_geo['cost'] = hexes[('cost_value', 'sum')]
hexes_geo['max_floors'] = hexes[('floors', 'max')]
hexes_geo['mean_floors'] = hexes[('floors', 'mean')]
hexes_geo['year_mean'] = hexes[('year_built', 'mean')]
hexes_geo['geometry'] = hexes_geo['id'].apply(lambda x: Polygon(h3.h3_to_geo_boundary(x, False)))

hexes_geo = gpd.GeoDataFrame(hexes_geo, geometry='geometry', crs=4326)
hexes_geo = hexes_geo.to_crs(3857)
hexes_geo = hexes_geo.reset_index(drop=True)

In [223]:
hexes_geo['year_max'] = hexes_geo['max_floors'].apply(lambda x: gdf[gdf['floors'] == x]['year_built'].min()).astype(int)

Вот, собственно говоря, те самые гексы, ради которых всё и затевалось: 

In [241]:
hexes_geo.explore()

Посчитаем ещё некоторые дополнительные параметры. В первую очередь - долю некоммерческой недвижимости от всей недвижимости, которая входит в гекс. 

In [225]:
purpose_groups = gdf.groupby('h')['purpose'].value_counts(normalize=True)
hexes_geo['commercial_ratio'] = [purpose_groups[(h, 'Нежилое')] for h in hexes_geo['id']]

Также посчитаем среднюю стоимость дома самым простым способом - поделим итоговую кадастровую стоимость на количество домов. Так как количество домов сильно разнится, средняя стоимость будет более адекватным показателем "дороговизны" имущества. 

In [226]:
hexes_geo['cost_mean'] = hexes_geo['cost'] / hexes_geo['buildings_count']

In [227]:
hexes_geo

,id,buildings_count,cost,max_floors,mean_floors,year_mean,geometry,year_max,commercial_ratio,cost_mean
0,872cce160ffffff,570,3.864451e+11,18,5,1946,"POLYGON ((4185144.085 7513482.780, 4183957.647...",1979,0.703509,6.779738e+08
1,872cce164ffffff,1606,1.480858e+12,19,4,1923,"POLYGON ((4186155.450 7508150.252, 4184969.327...",1999,0.735367,9.220788e+08
2,872cce165ffffff,536,4.408207e+11,39,5,1949,"POLYGON ((4187691.636 7512627.139, 4186505.167...",2018,0.716418,8.224266e+08
3,872cce166ffffff,1119,1.067231e+12,31,5,1940,"POLYGON ((4183608.334 7509006.776, 4182422.244...",1954,0.673816,9.537366e+08
4,872cce170ffffff,103,3.048329e+10,14,3,1968,"POLYGON ((4178514.901 7510718.245, 4177328.874...",1917,0.757282,2.959543e+08
5,872cce174ffffff,236,5.049558e+11,80,7,1964,"POLYGON ((4179526.002 7505392.810, 4178340.290...",2013,0.483051,2.139643e+09
6,872cce175ffffff,432,3.011344e+11,22,5,1954,"POLYGON ((4181061.484 7509862.774, 4179875.426...",1974,0.696759,6.970703e+08
7,872cce176ffffff,176,1.529099e+11,88,6,1974,"POLYGON ((4176979.854 7506249.161, 4175794.175...",2015,0.636364,8.688062e+08
8,872cce8c8ffffff,941,7.062743e+11,35,5,1931,"POLYGON ((4189713.248 7501962.138, 4188527.411...",2004,0.757705,7.505571e+08
9,872cce8c9ffffff,1656,8.789334e+11,33,4,1921,"POLYGON ((4191250.474 7506435.623, 4190064.290...",1953,0.755435,5.307569e+08


Нормализуем показатель стоимости - приведём его к такому виду, чтобы он варьировался в диапазоне от 0 до 1, где 0 - минимальная стоимость, 1 - максимальная. 

In [228]:
max_value = hexes_geo['cost_mean'].max()
min_value = hexes_geo['cost_mean'].min()

hexes_geo['cost_mean_normalized'] = hexes_geo['cost_mean'].apply(lambda x: (x - min_value) / (max_value - min_value))

Теперь подготовим данные к тому, чтобы воссоздать гексагоны в p5js. Для этого нормализуем положение каждого гекса по горизонтальной и вертикальной оси таким образом, чтобы среднее значение было равным 0, а модуль максимального и минимального значения - стандартному отклонению. 

Благодаря такому методу нормализации при работе в p5js мы сможем ориентироваться по этим показателям "от центра" картинки. 

In [229]:
hexes_geo['x'] = hexes_geo['geometry'].apply(lambda x: x.centroid.x)
hexes_geo['y'] = hexes_geo['geometry'].apply(lambda x: x.centroid.y)

In [230]:
hexes_geo['x_normalized'] = hexes_geo['x'].apply(lambda x: (x - hexes_geo['x'].mean()) / hexes_geo['x'].std())
hexes_geo['y_normalized'] = hexes_geo['y'].apply(lambda x: (x - hexes_geo['y'].mean()) / hexes_geo['y'].std())

In [231]:
hexes_geo = hexes_geo.sort_values(by=['x_normalized', 'y_normalized'])

In [232]:
hexes_geo.info()

<class 'geopandas.geodataframe.GeoDataFrame'>
Index: 24 entries, 7 to 20
Data columns (total 15 columns):
 #   Column                Non-Null Count  Dtype   
---  ------                --------------  -----   
 0   id                    24 non-null     object  
 1   buildings_count       24 non-null     int64   
 2   cost                  24 non-null     float64 
 3   max_floors            24 non-null     int64   
 4   mean_floors           24 non-null     int64   
 5   year_mean             24 non-null     int64   
 6   geometry              24 non-null     geometry
 7   year_max              24 non-null     int64   
 8   commercial_ratio      24 non-null     float64 
 9   cost_mean             24 non-null     float64 
 10  cost_mean_normalized  24 non-null     float64 
 11  x                     24 non-null     float64 
 12  y                     24 non-null     float64 
 13  x_normalized          24 non-null     float64 
 14  y_normalized          24 non-null     float64 
dtypes: fl

In [233]:
def normalize_geo(max_value, min_value, median_value, x):
    if x == median_value:
        return 0
    if x > median_value:
        return (x - median_value) / (max_value - median_value)
    
    return (x - median_value) / (min_value - median_value) * -1

In [234]:
hexes_geo['x_normalized_2'] = hexes_geo['x'].apply(lambda x: normalize_geo(hexes_geo.x.max(), hexes_geo.x.min(), hexes_geo.x.median(), x))
hexes_geo['y_normalized_2'] = hexes_geo['y'].apply(lambda x: normalize_geo(hexes_geo.y.max(), hexes_geo.y.min(), hexes_geo.y.median(), x))

In [235]:
hexes_geo[hexes_geo['y'] <= hexes_geo['y'].quantile(0.25)]

,id,buildings_count,cost,max_floors,mean_floors,year_mean,geometry,year_max,commercial_ratio,cost_mean,cost_mean_normalized,x,y,x_normalized,y_normalized,x_normalized_2,y_normalized_2
15,872cce8daffffff,249,1.872230e+11,14,4,1940,"POLYGON ((4180536.329 7500070.239, 4179350.932...",1917,0.763052,7.518996e+08,0.247301,4.180712e+06,7.503333e+06,-1.138010,-1.329480,-0.632583,-0.773197
13,872cce8d8ffffff,313,6.374254e+11,20,5,1956,"POLYGON ((4183082.572 7499211.957, 4181897.143...",1981,0.581470,2.036503e+09,0.944058,4.183258e+06,7.502476e+06,-0.686850,-1.521218,-0.369560,-0.886564
17,872cce8ddffffff,163,1.289526e+11,23,4,1933,"POLYGON ((4185629.081 7498353.148, 4184443.620...",1973,0.791411,7.911200e+08,0.268573,4.185805e+06,7.501618e+06,-0.235644,-1.713073,-0.106510,-1.000000
8,872cce8c8ffffff,941,7.062743e+11,35,5,1931,"POLYGON ((4189713.248 7501962.138, 4188527.411...",2004,0.757705,7.505571e+08,0.246572,4.189889e+06,7.505230e+06,0.487948,-0.905246,0.315178,-0.522366
12,872cce8cdffffff,438,3.148997e+11,24,5,1936,"POLYGON ((4192260.720 7501103.150, 4191074.852...",1967,0.751142,7.189491e+08,0.229428,4.192437e+06,7.504372e+06,0.939326,-1.097141,0.578194,-0.635825
23,872ccebb6ffffff,185,1.077797e+11,19,5,1956,"POLYGON ((4194808.455 7500243.635, 4193622.557...",1999,0.643243,5.825930e+08,0.155470,4.194985e+06,7.503514e+06,1.390750,-1.289154,0.841237,-0.749354


In [240]:
hexes_geo.sort_values(by='y').explore(
    column='y_normalized_2',
)

Сохраним итоговый результат 

In [237]:
hexes_summary = hexes_geo[['cost_mean_normalized', 'max_floors', 'mean_floors', 'year_max', 'year_mean', 'commercial_ratio', 'x_normalized', 'y_normalized', 'x_normalized_2', 'y_normalized_2']]

In [238]:
hexes_summary.to_dict(orient='records')

[{'cost_mean_normalized': 0.31070960452542745,
  'max_floors': 88,
  'mean_floors': 6,
  'year_max': 2015,
  'year_mean': 1974,
  'commercial_ratio': 0.6363636363636364,
  'x_normalized': -1.7682364255227758,
  'y_normalized': 0.05246117026397598,
  'x_normalized_2': -1.0,
  'y_normalized_2': 0.04383215352525372},
 {'cost_mean_normalized': 0.0,
  'max_floors': 14,
  'mean_floors': 3,
  'year_max': 1917,
  'year_mean': 1968,
  'commercial_ratio': 0.7572815533980582,
  'x_normalized': -1.4963141734244292,
  'y_normalized': 1.0524311207654213,
  'x_normalized_2': -0.8414714549930415,
  'y_normalized_2': 0.6343500374306894},
 {'cost_mean_normalized': 1.0,
  'max_floors': 80,
  'mean_floors': 7,
  'year_max': 2013,
  'year_mean': 1964,
  'commercial_ratio': 0.4830508474576271,
  'x_normalized': -1.3170936655155319,
  'y_normalized': -0.1388448352009454,
  'x_normalized_2': -0.7369873013978078,
  'y_normalized_2': -0.06922538969789611},
 {'cost_mean_normalized': 0.24730054771790552,
  'max_f

In [239]:
with open('data/data_visualization.json', 'w') as json_file:
    json.dump({'features': hexes_summary.to_dict(orient='records')}, json_file)